In [ ]:
import numpy as np
import pandas as pd


# Data Cleansing

## "Numbers" data

In [ ]:
numbers = pd.read_csv('boxoffice.csv')

In [ ]:
num_clean = numbers.copy()
num_clean = pd.DataFrame(num_clean)

In [ ]:
num_clean

In [ ]:
type(num_clean)

#### Domestic Box, International Box, Worldwide Box clean

In [ ]:
# Convert Domestic Box to be ready for data analysis
num_clean['domestic_box'] = num_clean['domestic_box'].str.lstrip('$')
num_clean['domestic_box'] = num_clean['domestic_box'].str.replace(',','')
num_clean['domestic_box'] = num_clean['domestic_box'].fillna(0)
num_clean['domestic_box'] = num_clean['domestic_box'].astype(int)

In [ ]:
# Convert International Box for data analysis
num_clean['international_box'] = num_clean['international_box'].str.lstrip('$')
num_clean['international_box'] = num_clean['international_box'].str.replace(',','')
num_clean['international_box'] = num_clean['international_box'].fillna(0)
num_clean['international_box'] = num_clean['international_box'].astype(int)

In [ ]:
#Convert Worldwide Box for data analysis
num_clean['worldwide_box'] = num_clean['worldwide_box'].str.lstrip('$')
num_clean['worldwide_box'] = num_clean['worldwide_box'].str.replace(',','')
num_clean['worldwide_box'] = num_clean['worldwide_box'].fillna(0)
num_clean['worldwide_box'] = num_clean['worldwide_box'].astype(int)

In [ ]:
# Correct for missing values on website. Worldwide = Domestic + International
num_clean['worldwide_box'] = num_clean['domestic_box'] + num_clean['international_box']

In [ ]:
num_clean

#### Production Budget clean

In [ ]:
#Copy dataset
num_clean2 = num_clean.copy()


In [ ]:
#Clean production budget for data analysis
num_clean2['production_budget'] = num_clean2['production_budget'].str.lstrip('$')
num_clean2['production_budget'] = num_clean2['production_budget'].str.replace(',','')
num_clean2['production_budget'] = num_clean2['production_budget'].str.split(' \(').str[0]
num_clean2['production_budget'] = num_clean2['production_budget'].str.replace('DNE','NaN')


In [ ]:
num_clean2

#### Release Date clean

In [ ]:
# Copy dataset
num_clean3 = num_clean2.copy()

In [ ]:
#Strip unnecessary text from date field
num_clean3['domestic_release'] = num_clean3['domestic_release'].str.split(' \(').str[0]

In [ ]:
# def datestrip(s):
#     return re.sub(r'')

#### Movie Title

In [ ]:
#Create copy
num_clean4 = num_clean3.copy()

In [ ]:
#Create two columns with original data
movieyear = num_clean4['movie_title'].str.split(' \(',n=1,expand=True)
movieyear[1] = movieyear[1].str.rstrip('\)')

In [ ]:
# Create column for movie name and year separately. Remove original column
num_clean4['movie_name'] = movieyear[0]
num_clean4['year'] = movieyear[1]
num_clean4.drop(columns = ['movie_title'], inplace=True)

In [ ]:
num_clean4

#### Running Time

In [ ]:
num_clean5 = num_clean4.copy()

In [ ]:
num_clean5['running_time'] = num_clean5['running_time'].str.replace('DNE','NaN')
num_clean5['running_time'] = num_clean5['running_time'].str.split(' ').str[0]


In [ ]:
num_clean5

In [ ]:
num_clean5.groupby('source')['international_box','worldwide_box'].describe()

## RottenTomatoes data

In [ ]:
rotten = pd.read_csv('rottentomatoes.csv')

In [ ]:
rottenclean = rotten.copy()
rottenclean

#### User review count

In [ ]:
# Remove "User Ratings:"
rottenclean['count_user_review'] = rottenclean['count_user_review'].str.split(':').str[1]
#Remove "Not yet available"
rottenclean['count_user_review'] = rottenclean['count_user_review'].str.replace('Not yet available','0')
#Remove commas
rottenclean['count_user_review'] = rottenclean['count_user_review'].str.replace(',','')
#Convert to int
rottenclean['count_user_review'] = rottenclean['count_user_review'].astype(int)

# Convert "Not yet available" which was '0' back to integers
rottenclean['count_user_review'] = rottenclean['count_user_review'].replace(0,'NaN')


In [ ]:
rottenclean

#### Convert all other necessary columns to integers

In [ ]:
rottenclean2 = rottenclean.copy()

In [ ]:
#Convert critic score to int
rottenclean2['critic_score'] = rottenclean2['critic_score'].str.rstrip('\%')
rottenclean2['critic_score'] = rottenclean2['critic_score'].dropna().astype(int)

#Convert count_critic_review to int
rottenclean2['count_critic_review'] = rottenclean2['count_critic_review'].dropna().astype(int)

#Convert user_score to int
rottenclean2['user_score'] = rottenclean2['user_score'].str.rstrip('\%')
rottenclean2['user_score'] = rottenclean2['user_score'].dropna().astype(int)




# rottenclean2['count_critic_review'] = rottenclean2['count_critic_review'].astype(int)

In [ ]:
rottenclean2

In [ ]:
rottenclean2.groupby(['certified_status'])['critic_score','user_score'].describe()

## Combine "Numbers" and "RottenTomatoes" tables

rottenclean2  - movie_name
num_clean5 - movie_name

In [ ]:
RT_an = rottenclean2.copy()
Num_an = num_clean5.copy()

In [ ]:
rotten_clean2.columns()

In [ ]:
total_data = pd.merge(Num_an,RT_an, how = 'left', left_on='movie_name', right_on='movie_name')

In [ ]:
total_data.groupby('certified_status')['critic_score','user_score'].describe()

In [ ]:
total_data.to_csv('total_data.csv')

In [ ]:
type(total_data.production_country[1])

#### Production Country split (Production Country analysis)

Create a "branch" of the dataset to do Product Country analysis

In [ ]:
# Make copy of aggregated dataframe
total_data2 = total_data.copy()

In [ ]:
total_data2.production_country.head()

In [ ]:
list(enumerate(total_data2.production_country))

In [ ]:
list_ = []
for index, item in enumerate(total_data2.production_country):
    list_.extend(map(lambda x: [index, x], item))
    
prod_coun_df = pd.DataFrame(list_,columns=['index','production_countries'])

In [ ]:
prod_coun = pd.merge(total_data2, prod_coun_df, how='right',left_index=True,right_on='index')

In [ ]:
prod_coun = prod_coun.drop('production_country', axis=1)

In [ ]:
prod_coun.head(40)

#### Language split (Language analysis split)

Don't forget to default back to previous complete dataframe.

Need to create a separate "branch" for data analysis

In [ ]:
langs = total_data2.copy()

In [ ]:
langs.language = langs.language.apply(lambda x: x.split(','))

In [ ]:
list(enumerate(langs.language))

In [ ]:
list_ = []
for index, item in enumerate(langs.language):
    list_.extend(map(lambda x: [index, x], item))
    
lang_df = pd.DataFrame(list_,columns=['index','languages'])

In [ ]:
lang_df = pd.merge(langs, lang_df, how='right',left_index=True,right_on='index')

In [ ]:
lang_df.head(40)